In [ ]:
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
import torch

In [ ]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)
model = LlamaForCausalLM.from_pretrained(
    MODEL_NAME,
    return_dict=True,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config

In [ ]:
model = model.eval()


In [ ]:
prompt = "My name is Vikas"
# prompt = "Names of my friends are: Anmol, Aryan, Joel. How many friends do I have?"
prompt = "Do you recall the names of my three friends?"


In [ ]:
encoding = tokenizer(prompt, return_tensors="pt").to(model.device)

In [ ]:
print(encoding)
print(encoding.input_ids)
print(encoding.input_ids.shape)

In [ ]:
# %%time
with torch.no_grad():
    outputs = model.generate(
        **encoding,
        max_new_tokens=256,
        temperature=0.0001,
        generation_config=generation_config,
    )

In [ ]:
print("len", len(outputs))
print("shape", outputs.shape)



In [ ]:

output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

In [ ]:
answer_tokens = outputs[:, encoding.input_ids.shape[1] :]


print("Original shape", outputs.shape)
print("Inpute Shape", encoding.input_ids.shape)
print("New shape", answer_tokens.shape)



output_text = tokenizer.decode(answer_tokens[0], skip_special_tokens=True).strip()
print(output_text)

In [ ]:
def format_prompt_old(prompt: str, system_prompt: str) -> str:
    return f"""
{system_prompt}
{prompt}
""".strip()

In [ ]:
format_prompt("vikas","verma")

In [ ]:
def generate_response(prompt: str, max_new_tokens: int = 128, temperature: float = 0.001) -> str:
    encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model.generate(
            **encoding,
            max_new_tokens=max_new_tokens,
            temperature=0.001,
            generation_config=generation_config,

            do_sample = True,
        )
    answer_tokens = outputs[:, encoding.input_ids.shape[1] :]
    return tokenizer.decode(answer_tokens[0], skip_special_tokens=True)
     

In [ ]:
l = generate_response(prompt, max_new_tokens =  128, temperature = 0.001)
print(l)

In [ ]:
prompt = """
Write an email to a director flaunting about your movie career
""".strip()
response = generate_response(format_prompt(prompt, SYSTEM_PROMPT), max_new_tokens=512)
print(response)

In [ ]:
format_prompt(prompt, SYSTEM_PROMPT)

In [ ]:
prompt = """
Fake News is becoming a huge problem today.
""".strip()
responses = []
for _ in range(10):
    response = generate_response(format_prompt(prompt, SYSTEM_PROMPT), max_new_tokens=200)
    print(response)
    SYSTEM_PROMPT = "You know that " + response

In [ ]:
SYSTEM_PROMPT = """
Write a response that answers the request according to the conversation history below. Answer in one sentence only.
"""
# prompt = input("Enter First Input")
initial_length = len(SYSTEM_PROMPT)
prompt = "Hi."



firstgo = True
noresponse = False
i = 0
errorCount = 0
while i < 10:
    print(f"i {i} errorCount {errorCount}")
    if errorCount > 10:
        print("Too many errors, breaking out.")
        break
    if firstgo:
        SYSTEM_PROMPT+='\nRequest: '+prompt
        firstgo = False
    elif not noresponse:
        prompt = input("Enter Reply")
    else:
        pass

    response = generate_response(format_prompt(prompt, SYSTEM_PROMPT), max_new_tokens=200, temperature= 0.001)

    # if len(response) < 3:
    #     noresponse = True
    #     errorCount += 1
    #     continue
    # else:
    #     noresponse = False
    print("Request: ", prompt)
    print("Response:", response)
    SYSTEM_PROMPT+='\nResponse: '+ response
    i += 1
    

In [ ]:
response

In [ ]:
print(SYSTEM_PROMPT)

In [ ]:
a = "Hi"
b = """
Write a response that answers the request according to the conversation history below. Answer in one sentence only.
"""
b +='\nRequest: '+prompt
print(format_prompt(a, b))

In [ ]:
theinput = """<s>[INST] <<SYS>>
{{ 
    Write a response that answers the request according to the conversation history below. Answer in one sentence only.\nRequest: Hello AI, My name is Vikas, what is your name?\nResponse: Hello Vikas! My name is LLaMA, I'm a large language model trained by a team of researcher at Meta AI. How are you today?\n
}}
<</SYS>>

xxx [/INST] yyy </s><s>[INST] xxx [/INST] yyy </s><s>[INST] xxx [/INST] yyy </s><s>[INST] {{ I am very good. The day is sunny, how is the weather there where you are? }} [/INST]"""
generate_response(theinput, 200, 0.001)

In [ ]:
theinput = """<s>[INST] <<SYS>>
{{ 
    Write a response that answers the request according to the conversation history below. Answer in one short sentence only.
    Request: Hello AI, My name is Vikas, what is your name?
    Response: Hello Vikas! My name is LLaMA, I'm a large language model trained by a team of researcher at Meta AI. How are you today?
    Request: I am very good. Where do you live? 
    Response: Hello Vikas! I'm just an AI, I don't have a physical body or a specific location where I live. I exist in the form of computer code and data, and I can be accessed from anywhere in the world where there is an internet connection.
}}
<</SYS>>

 [/INST]  </s><s>[INST]  [/INST]  </s><s>[INST]  [/INST]  </s><s>[INST] {{ But I am using you without internet }} [/INST]"""
generate_response(theinput, 200, 0.001)

In [ ]:
def format_prompt(prompt, history=''):
    return f"""<s>[INST] <<SYS>>
{{ 
    Write a response that answers the request according to the conversation history below. Answer in one short sentence only.
    {history}
}}
<</SYS>>

 [/INST]  </s><s>[INST]  [/INST]  </s><s>[INST]  [/INST]  </s><s>[INST] {{ {prompt} }} [/INST]"""

In [ ]:
theinput = """<s>[INST] <<SYS>>
{{ 
    Write a response that answers the request according to the conversation history below. Answer in one short sentence only.

}}
<</SYS>>

 [/INST]  </s><s>[INST]  [/INST]  </s><s>[INST]  [/INST]  </s><s>[INST] {{ But I am using you without internet }} [/INST]"""
generate_response(theinput, 200, 0.001)

In [ ]:
prompt = 'Fake News is a real problem. What do you think about it?'
import sys
history = '''Request: Hi
Response:  Hello!
'''
for _ in range(50):

    print(history)
    print("*" * 50)
    while True:
        prompt = input()
        if len(prompt) == 0:
            continue
        else:
            break
    if prompt == 'quit':
        break
    the_output = generate_response(format_prompt(prompt, history), 200, 0.001)
    history += "Request: " + prompt + '\n'
    history += "Response: " + the_output + '\n\n'

In [ ]:
print(history)